In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import transformers

In [ ]:
# transformers.__version__

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
from datasets import get_dataset_config_names
from google.colab import userdata
key_ = userdata.get('HF_TOKEN')


In [4]:
import polars as pl
import pandas as pd
import joblib

In [5]:
df = pd.read_csv("/content/drive/MyDrive/essay/data/train.csv")

In [7]:
df.shape

(17307, 3)

In [ ]:
lbl = sorted(list(df.score.unique()))

lbl

[i for i, j in enumerate(lbl)]


lbl_map = {j: i for i, j in enumerate(lbl)}
lbl_map
df_map = (
    df
    .assign(
       score = lambda f: f.score.map(lambda p: lbl_map[p])
                          .astype(int)
    )
)


In [ ]:

df_map

In [ ]:
from datasets import Dataset, DatasetDict, Features, ClassLabel, Value


features = Features({
    'essay_id': Value('string'),
    'full_text': Value('string'),
    'score': ClassLabel(names = [int(i) for i in  lbl],
                            id = [int(i) for i, j in enumerate(lbl)]
                            )
})


def create_dataset(df):
    essay_id = df.essay_id.to_list()
    full_text = df.full_text.to_list()
    score = df.score.tolist()


    return Dataset.from_dict({
        'essay_id': essay_id,
        'full_text': full_text,
        'score': score
    }, features=features)


def create_fold(train, vali, test):
    return DatasetDict({
        'train': create_dataset(train),
        'validation': create_dataset(vali),
        'test': create_dataset(test)
    })




In [ ]:

create_dataset(df_map).push_to_hub("essay_train_full", token =userdata.get('HF_TOKEN') )

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/prl90777/essay_train_full/commit/e00008f7b0f9d879e353b59701dca78960379186', commit_message='Upload dataset', commit_description='', oid='e00008f7b0f9d879e353b59701dca78960379186', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

num_folds = 5
seed = 0
shuffle = True

df_pd = df_map
for i in [0, 50, 100, 150]:
  seed  = i
  stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=shuffle, random_state=seed)
  for fold, (train_index, test_index) in enumerate(stratified_kfold.split(df_pd, df_pd['score'])):
      desc = f"essay_{seed}_{fold}"
      print(desc)
      train_index, val_index = train_test_split(train_index, test_size=0.2, random_state=seed)

      train = df_pd.iloc[train_index]
      vali = df_pd.iloc[val_index]
      test = df_pd.iloc[test_index]
      dataset_folds = create_fold(train, vali, test)
      dataset_folds.push_to_hub(desc, token =userdata.get('HF_TOKEN') )


essay_0_0


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/783 [00:00<?, ?B/s]

In [ ]:
dataset_folds['train'][:5]

{'essay_id': ['1ced533', '71264f4', '7736261', 'b0fc064', 'c48a5bd'],
 'full_text': ['Technology that can make Mona Lisa smile. Facial Action Coding System enables computers to identify human emotions. It is able to read a person emotional state. This technology can be useful in many was. In classrooms it technology can improve learning.\n\nFacial Action Coding System can improve a child\'s learning. It could recognize when a student is becoming confused or bored. This can help modify the lesson so the student can stay forcesed on the lesson. That would help a lot of student work at the best speed for them. The Facial Action Coding would work by watching the muscles in a person\'s face to tell what mood their in. Even if the Computer couldn\'t handle the complex algorithms used. They can weite down some simple instructions that "encode" different emotions. This could really help keep student on key with their learning.\n\nThis technology can be useful in many was. In classrooms it tech

In [ ]:

from datasets import load_dataset
squad_dataset = load_dataset("prl90777/essay_150_0", token = userdata.get('HF_TOKEN'))


Generating train split:   0%|          | 0/11076 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2769 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [ ]:
squad_dataset['train'][0:2]

{'essay_id': ['1ca4a2d', 'df0a3ec'],
 'full_text': ['The author of "The challenge of exploring space" does a excellent job supporting the arugument that Venus should be further explored. They did this by using research done by NASA, scientific projects, and even presenting challenge to the reader or audience.\n\nIn pararaph 5 the author intoroduces an idea that NASA has made special products to explore the face of Venus. The author says " imagine a blimp-like vehicle hovering 30 or so miles above the roiing venusian landscape. Just as our jet airplanes travel at a high altitude to fly over many storms, a vehice hovering over venus woud avoid the unfriendly ground conditions by staying up and out of their way." This evidence helps convey the idea that the face of venus shoud further be explored because space experts have been working on it.\n\nSimilarly, in paragaph 7 the author intorduces a new idea that NASA has also been working on. Electronics that can survive the harsh enviornment 

In [ ]:
hug_df['fold_0']['train'].features['db_source']

ClassLabel(names=['orginal', 'moth', 'nicholas'], id=[0, 1, 2])

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from huggingface_hub import HfFolder, Repository
import os

def create_fold(train, validation, test, fold_dir):
    os.makedirs(fold_dir, exist_ok=True)
    train.to_parquet(os.path.join(fold_dir, 'train.parquet'))
    validation.to_parquet(os.path.join(fold_dir, 'validation.parquet'))
    test.to_parquet(os.path.join(fold_dir, 'test.parquet'))

num_folds = 5
df_pd = df_map

for seed in [0, 50, 100, 150]:
    stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    for fold, (train_index, test_index) in enumerate(stratified_kfold.split(df_pd, df_pd['score'])):
        desc = f"essay_{seed}_{fold}"
        print(desc)
        train_index, val_index = train_test_split(train_index, test_size=0.2, random_state=seed)

        train = df_pd.iloc[train_index]
        vali = df_pd.iloc[val_index]
        test = df_pd.iloc[test_index]

        fold_dir = f'/content/{desc}'
        create_fold(train, vali, test, fold_dir)

        # Initialize and update repository
        repo = Repository(local_dir=fold_dir, clone_from=f'https://huggingface.co/{username}/{repo_name}')
        repo.git_add(auto_lfs_track=True)
        repo.git_commit(f"Add fold {fold} with seed {seed}")
        repo.git_push()
